## PwC's replication of PD Tranisition matrices at proptfolio level and at segment levels: 

In [1]:
#import the required libraries: 
import pandas as pd
import numpy as np

In [10]:
## Read the cleaned and processed PD data:
df_cleaned=pd.read_csv("//rasscvo/DIAC/DIAC/Farm_Credit_System/From_PwC/AgWest/2024/Development Materials/Model_Development/Outputs/PD/cleaned_PD_data_w_segmentation.csv")
display(df_cleaned.head())
print ('Dimension of the cleaned file:', df_cleaned.info())

C:\Users\dghatak005\AppData\Local\Temp\ipykernel_1448\3818159093.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned=pd.read_csv("//rasscvo/DIAC/DIAC/Farm_Credit_System/From_PwC/AgWest/2024/Development Materials/Model_Development/Outputs/PD/cleaned_PD_data_w_segmentation.csv")


,Date,UniqueID,EmpowerPDTemplate,State,FarmType,FCLoanType,PDRatingFinal,SICCode1Description,AssociationNumber,ProductTypeDescription,...,ever_default_flag,next_year,maturity_flag,payoff_flag,end_PD_rating,initial_PD_rating,initial_rating_year,initial_rating_missing_flag,Farm.Type.Description..AgWest.,Segment
0,2015-12-31,S0000071001,South,California,NOT APPLICABLE,NOT APPLICABLE,6,All Other Miscellaneous Crop Farming,146,CONDITIONAL SALE,...,False,2016.0,False,False,6,NaN,NaN,True,Diversified Crops,Other Ag
1,2016-12-31,S0000071001,South,California,NOT APPLICABLE,NOT APPLICABLE,6,All Other Miscellaneous Crop Farming,146,CONDITIONAL SALE,...,False,2017.0,False,False,6,6,2015.0,False,Diversified Crops,Other Ag
2,2017-11-30,S0000071001,South,California,NOT APPLICABLE,NOT APPLICABLE,6,All Other Miscellaneous Crop Farming,146,CONDITIONAL SALE,...,False,NaN,True,False,Matured,6,2016.0,False,Diversified Crops,Other Ag
3,2016-12-31,S0000071002,South,California,NOT APPLICABLE,NOT APPLICABLE,6,All Other Miscellaneous Crop Farming,146,CONDITIONAL SALE,...,False,2017.0,False,False,6,NaN,NaN,True,Diversified Crops,Other Ag
4,2017-12-31,S0000071002,South,California,NOT APPLICABLE,NOT APPLICABLE,6,All Other Miscellaneous Crop Farming,146,CONDITIONAL SALE,...,False,2018.0,False,False,6,6,2016.0,False,Diversified Crops,Other Ag


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398883 entries, 0 to 398882
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               398883 non-null  object 
 1   UniqueID                           398883 non-null  object 
 2   EmpowerPDTemplate                  398883 non-null  object 
 3   State                              398883 non-null  object 
 4   FarmType                           398582 non-null  object 
 5   FCLoanType                         398883 non-null  object 
 6   PDRatingFinal                      398883 non-null  int64  
 7   SICCode1Description                398558 non-null  object 
 8   AssociationNumber                  398883 non-null  int64  
 9   ProductTypeDescription             398883 non-null  object 
 10  LoanMaturity                       398883 non-null  object 
 11  LoanSizeAtOrigination              3988

In [16]:
# List desired segments from Segment column in cleaned PD data  
segments = ["Dairy & Animal Products", "Home Mortgage & CHL", "Wine & Vineyard", "Tree Nuts", "Ag Services", "Other Ag"] 
# List of order of potential ratings from least risk to highest risk  
rating_order = ['1', '2', '3', 'Low Risk', '4', '5', '6 to 9', '6', '7', '8', '9', '10', '11', 'Default', 'Matured', 'Paid Off']  

In [17]:
## Filter out and apply relevant exclusions:
df_processed = df_cleaned[~df_cleaned['initial_PD_rating'].isna()]  
df_processed = df_processed[df_processed['initial_PD_rating'] != "Default"]  
df_processed = df_processed[df_processed['Segment'].isin(segments)]

print("Lenght of the data after exclusions:", len(df_processed))

Lenght of the data after exclusions: 310233


In [126]:
  
def generate_transition_matrices(df, region, segments, return_option, exclude_maturity_payoff=True):  
    # Function to generate single-year transition matrix output

    if region == 'Combined':  
        matrix_data = df  
    else:  
        matrix_data = df[df['EmpowerPDTemplate'] == region]  
  
  
    if exclude_maturity_payoff:  
        matrix_data = matrix_data[~matrix_data['end_PD_rating'].isin(['Matured', 'Paid off'])]  
  
    table = matrix_data.groupby(['Segment', 'year', 'initial_PD_rating', 'end_PD_rating']).agg(  
        count_by=('Balance', 'count') 
    ).reset_index()  
  
    value_col = "count_by"  
  
  
    table = table.pivot_table(index=['Segment', 'year', 'initial_PD_rating'], columns='end_PD_rating', values=value_col, fill_value=0)  
    table = table.reindex(sorted(table.columns, key=lambda x: rating_order.index(x)), axis=1)  
  
    table['Total'] = table.sum(axis=1)  
  
    table.reset_index(inplace=True)  
   
    table['Region'] = region  


      
    if return_option == 'proportions':  
        table_props = table.copy()  
        table_props.iloc[:, 3:13] = table_props.iloc[:, 3:13].div(table_props['Total'], axis=0)  
        
        # table2 = table_props.div(table_props['Total'], axis=0) 
        table2=table_props.copy()
        # return table_props  
    elif return_option == 'counts':  
        table2=table.copy()
        # return sorted_df  

    grouped_df = table2.groupby(['Segment','year'])  
  
# Sort within each group based on the custom order  
    sorted_df = grouped_df.apply(lambda x: x.sort_values('initial_PD_rating', key=lambda y: pd.Categorical(y, categories=rating_order, ordered=True)))  
  
# Reset the index of the sorted DataFrame  
    sorted_df = sorted_df.reset_index(drop=True)  
    return sorted_df
  
  

In [128]:
# Generate single-year transition matrices for all segments by region  
north_count = generate_transition_matrices(df_processed, "North", segments, "counts", True)  
south_count = generate_transition_matrices(df_processed, "South", segments, "counts", True)  
combined_count = generate_transition_matrices(df_processed, "Combined", segments, "counts", True)  

In [129]:
## concat dataframes:
frames=[north_count,south_count,combined_count]
final_stacked_ttm_count=pd.concat(frames)
final_stacked_ttm_count

end_PD_rating,Segment,year,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Total,Region
0,Ag Services,2014,Low Risk,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,North
1,Ag Services,2014,5,0.0,61.0,7.0,3.0,3.0,1.0,0.0,0.0,0.0,75.0,North
2,Ag Services,2014,6,2.0,20.0,63.0,3.0,1.0,0.0,0.0,0.0,0.0,89.0,North
3,Ag Services,2014,7,0.0,6.0,13.0,56.0,5.0,0.0,0.0,0.0,0.0,80.0,North
4,Ag Services,2014,8,0.0,0.0,4.0,7.0,40.0,4.0,0.0,0.0,0.0,55.0,North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,Wine & Vineyard,2022,7,0.0,4.0,48.0,293.0,19.0,1.0,0.0,0.0,0.0,365.0,Combined
586,Wine & Vineyard,2022,8,0.0,0.0,13.0,48.0,221.0,43.0,6.0,1.0,0.0,332.0,Combined
587,Wine & Vineyard,2022,9,0.0,0.0,0.0,0.0,20.0,75.0,4.0,6.0,0.0,105.0,Combined
588,Wine & Vineyard,2022,10,0.0,0.0,0.0,0.0,5.0,20.0,17.0,3.0,0.0,45.0,Combined


In [130]:
# Generate single-year transition matrices for all segments by region  
north_prop = generate_transition_matrices(df_processed, "North", segments, "proportions", True)  
south_prop = generate_transition_matrices(df_processed, "South", segments, "proportions", True)  
combined_prop = generate_transition_matrices(df_processed, "Combined", segments, "proportions", True)  

In [131]:
## concat dataframes:
frames=[north_prop,south_prop,combined_prop]
final_stacked_ttm_prop=pd.concat(frames)
final_stacked_ttm_prop

end_PD_rating,Segment,year,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Total,Region
0,Ag Services,2014,Low Risk,0.916667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,North
1,Ag Services,2014,5,0.000000,0.813333,0.093333,0.040000,0.040000,0.013333,0.000000,0.000000,0.000000,1.0,North
2,Ag Services,2014,6,0.022472,0.224719,0.707865,0.033708,0.011236,0.000000,0.000000,0.000000,0.000000,1.0,North
3,Ag Services,2014,7,0.000000,0.075000,0.162500,0.700000,0.062500,0.000000,0.000000,0.000000,0.000000,1.0,North
4,Ag Services,2014,8,0.000000,0.000000,0.072727,0.127273,0.727273,0.072727,0.000000,0.000000,0.000000,1.0,North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,Wine & Vineyard,2022,7,0.000000,0.010959,0.131507,0.802740,0.052055,0.002740,0.000000,0.000000,0.000000,1.0,Combined
586,Wine & Vineyard,2022,8,0.000000,0.000000,0.039157,0.144578,0.665663,0.129518,0.018072,0.003012,0.000000,1.0,Combined
587,Wine & Vineyard,2022,9,0.000000,0.000000,0.000000,0.000000,0.190476,0.714286,0.038095,0.057143,0.000000,1.0,Combined
588,Wine & Vineyard,2022,10,0.000000,0.000000,0.000000,0.000000,0.111111,0.444444,0.377778,0.066667,0.000000,1.0,Combined


In [266]:
final_stacked_trans_matrc.to_excel("//rasscvo/DIAC/DIAC/Farm_Credit_System/AgWest/2024/PwC_Repl_Output/PD/replicated_ttm_all.xlsx")


In [267]:
final_stacked_ttm_prop.to_excel("//rasscvo/DIAC/DIAC/Farm_Credit_System/AgWest/2024/PwC_Repl_Output/PD/replicated_ttm_all_prp.xlsx")

## Segment Level Historical Avg Transition matrices: 

In [168]:
## Segment wise avg matrix from 2010-2019:
all_segment_avg= pd.DataFrame()
segments = ["Ag Services", "Dairy & Animal Products", "Home Mortgage & CHL",  "Other Ag", "Tree Nuts", "Wine & Vineyard"] 

for segment in segments:
    print(segment)  
  
    # Input data  
    input_data = final_stacked_ttm_prop[(final_stacked_ttm_prop['Segment'] == segment) & (final_stacked_ttm_prop['Region'] == 'Combined')].copy() 
    input_data=input_data[(input_data['year']>=2010 ) & (input_data['year']<=2019 )]  ## To filter out periods out side 2010-2019
    input_data.reset_index(drop=True, inplace=True)  
    input_data = pd.DataFrame(input_data)
    seg_avg=input_data.groupby('initial_PD_rating').apply(lambda x: x.iloc[:, 3:12].mean(axis=0)).reset_index()
    seg_avg['Segment']=segment

    # Sort within each group based on the custom order  
    grouped_df = seg_avg.groupby(['Segment'])  
    seg_avg_sorted = grouped_df.apply(lambda x: x.sort_values('initial_PD_rating', key=lambda y: pd.Categorical(y, categories=rating_order, ordered=True)))  
    # Reset the index of the sorted DataFrame  
    seg_avg_sorted = seg_avg_sorted.reset_index(drop=True) 

    #Stack segment avg datframes:
    all_segment_avg=pd.concat([all_segment_avg,seg_avg_sorted])   
    


all_segment_avg

Ag Services
Dairy & Animal Products
Home Mortgage & CHL
Other Ag
Tree Nuts
Wine & Vineyard


end_PD_rating,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment
0,Low Risk,0.796018,0.099704,0.053620,0.047210,0.003448,0.000000,0.000000,0.000000,0.000000,Ag Services
1,5,0.029338,0.793672,0.133479,0.033274,0.005728,0.003095,0.001414,0.000000,0.000000,Ag Services
2,6,0.002655,0.056136,0.742698,0.147711,0.037518,0.008765,0.000843,0.002041,0.001633,Ag Services
3,7,0.001588,0.014885,0.060338,0.797308,0.088064,0.026977,0.001424,0.006959,0.002457,Ag Services
4,8,0.000748,0.005321,0.039410,0.123465,0.711275,0.105691,0.007871,0.003435,0.002785,Ag Services
5,9,0.000000,0.000917,0.004012,0.051471,0.079883,0.762324,0.021774,0.023691,0.055926,Ag Services
6,10,0.000000,0.000000,0.000000,0.033333,0.041026,0.104418,0.672910,0.128215,0.020098,Ag Services
7,11,0.000000,0.000000,0.000000,0.000000,0.032317,0.028825,0.124592,0.732003,0.082263,Ag Services
0,Low Risk,0.810233,0.116623,0.038006,0.023193,0.003861,0.007895,0.000189,0.000000,0.000000,Dairy & Animal Products
1,5,0.047143,0.746296,0.117930,0.053598,0.025907,0.003666,0.005000,0.000460,0.000000,Dairy & Animal Products


In [268]:
all_segment_avg.to_excel("//rasscvo/DIAC/DIAC/Farm_Credit_System/AgWest/2024/PwC_Repl_Output/PD/all_segment_avg.xlsx")

## Portfolio level  wtd avg matrix calculation :

In [216]:
## First let's calculate the segment level weights: 
dairy_segments= ["Ag Services", "Dairy & Animal Products",  "Other Ag", "Tree Nuts", "Wine & Vineyard"] 
df_wts_1=df_processed[['EmpowerPDTemplate', 'Segment', 'Balance','PrincipalBalanceNetOfPartSold','year','Date','month']]

# Exclude CHL segment for Weights calcualation:
df_wts_2 = df_wts_1[df_wts_1['Segment'].isin(dairy_segments)]
df_wts_3= df_wts_2[(df_wts_2['year']==2022 ) & (df_wts_2['month']==12 )]  ## To filter out periods out side 2010-2019


df_wts_3.head()

,EmpowerPDTemplate,Segment,Balance,PrincipalBalanceNetOfPartSold,year,Date,month
39,South,Other Ag,23925.67,23925.67,2022,2022-12-31,12
132,South,Other Ag,3410317.45,3410317.45,2022,2022-12-31,12
551,South,Ag Services,994.31,994.31,2022,2022-12-31,12
555,South,Ag Services,3784.37,3784.37,2022,2022-12-31,12
559,South,Ag Services,2977.63,2977.63,2022,2022-12-31,12


In [217]:
df_wts_grouped=df_wts_3.groupby('Segment').agg({'PrincipalBalanceNetOfPartSold':'sum','Segment':'count'}).rename(columns={'PrincipalBalanceNetOfPartSold':'Total_Balance', 'Segment':'Count'}).reset_index()
df_wts_grouped['Weight']=df_wts_grouped['Total_Balance']/df_wts_grouped['Total_Balance'].sum()
df_wts_grouped

,Segment,Total_Balance,Count,Weight
0,Ag Services,1.037986e+09,1418,0.047060
1,Dairy & Animal Products,4.604598e+09,5173,0.208761
2,Other Ag,1.193604e+10,13298,0.541151
3,Tree Nuts,2.768639e+09,2207,0.125523
4,Wine & Vineyard,1.709521e+09,1419,0.077505


In [259]:
## since the weights are not matchign with the developer ones; we are directly useing their weights:


wt_data_agwst = {'Segment': ['Ag Services', 'Dairy & Animal Products', 'Other Ag','Tree Nuts', 'Wine & Vineyard'],  
        'Weight': [0.055294906,0.202180289,0.545291782,0.125356157,0.071876866]} 

# Create the DataFrame  
df_wts_grouped_agwst = pd.DataFrame(wt_data_agwst)  
df_wts_grouped_agwst




,Segment,Weight
0,Ag Services,0.055295
1,Dairy & Animal Products,0.202180
2,Other Ag,0.545292
3,Tree Nuts,0.125356
4,Wine & Vineyard,0.071877


In [260]:
# all_seg_avg_wt=all_segment_avg.merge(df_wts_grouped[['Weight','Segment']], how='left', on='Segment') 
##Usinh AgWest Numbers:
all_seg_avg_wt=all_segment_avg.merge(df_wts_grouped_agwst[['Weight','Segment']], how='left', on='Segment') 
all_seg_avg_wt=all_seg_avg_wt[all_seg_avg_wt['Segment']!='Home Mortgage & CHL']
all_seg_avg_wt.head()

,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment,Weight
0,Low Risk,0.796018,0.099704,0.053620,0.047210,0.003448,0.000000,0.000000,0.000000,0.000000,Ag Services,0.055295
1,5,0.029338,0.793672,0.133479,0.033274,0.005728,0.003095,0.001414,0.000000,0.000000,Ag Services,0.055295
2,6,0.002655,0.056136,0.742698,0.147711,0.037518,0.008765,0.000843,0.002041,0.001633,Ag Services,0.055295
3,7,0.001588,0.014885,0.060338,0.797308,0.088064,0.026977,0.001424,0.006959,0.002457,Ag Services,0.055295
4,8,0.000748,0.005321,0.039410,0.123465,0.711275,0.105691,0.007871,0.003435,0.002785,Ag Services,0.055295


In [261]:
#multiply the weights for each segments: 

all_seg_avg_wt_2=all_seg_avg_wt.copy()
all_seg_avg_wt_2.iloc[:, 1:10] = all_seg_avg_wt_2.iloc[:, 1:10].mul(all_seg_avg_wt_2['Weight'], axis=0)  
display(all_seg_avg_wt_2.head())

,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment,Weight
0,Low Risk,0.044016,0.005513,0.002965,0.002610,0.000191,0.000000,0.000000,0.000000,0.000000,Ag Services,0.055295
1,5,0.001622,0.043886,0.007381,0.001840,0.000317,0.000171,0.000078,0.000000,0.000000,Ag Services,0.055295
2,6,0.000147,0.003104,0.041067,0.008168,0.002075,0.000485,0.000047,0.000113,0.000090,Ag Services,0.055295
3,7,0.000088,0.000823,0.003336,0.044087,0.004870,0.001492,0.000079,0.000385,0.000136,Ag Services,0.055295
4,8,0.000041,0.000294,0.002179,0.006827,0.039330,0.005844,0.000435,0.000190,0.000154,Ag Services,0.055295


In [262]:
## Group and sumproduct:
all_seg_avg_wt_3=all_seg_avg_wt_2.groupby('initial_PD_rating').apply(lambda x: x.iloc[:, 1:10].sum(axis=0)).reset_index()
all_seg_avg_wt_3['Segment']='Full_Population'

# Sort within each group based on the custom order  
grouped_df = all_seg_avg_wt_3.groupby(['Segment'])  
pop_wtd_ttm_sorted = grouped_df.apply(lambda x: x.sort_values('initial_PD_rating', key=lambda y: pd.Categorical(y, categories=rating_order, ordered=True)))  
# Reset the index of the sorted DataFrame  
pop_wtd_ttm_sorted = pop_wtd_ttm_sorted.reset_index(drop=True) 
pop_wtd_ttm_sorted

    

,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment
0,Low Risk,0.817340,0.116970,0.040001,0.018267,0.002910,0.003143,0.000766,0.000270,0.000333,Full_Population
1,5,0.036077,0.792832,0.111087,0.041826,0.012501,0.002844,0.001779,0.000550,0.000504,Full_Population
2,6,0.009023,0.074051,0.749675,0.113385,0.036202,0.010484,0.003763,0.002630,0.000787,Full_Population
3,7,0.003328,0.020233,0.085470,0.762989,0.087591,0.024666,0.007912,0.005524,0.002288,Full_Population
4,8,0.001250,0.008110,0.032735,0.137167,0.689681,0.086040,0.027388,0.015771,0.001857,Full_Population
5,9,0.000141,0.005265,0.019534,0.046942,0.106801,0.704820,0.062924,0.038517,0.015056,Full_Population
6,10,0.000449,0.000343,0.004201,0.019583,0.104911,0.140472,0.511698,0.195508,0.022835,Full_Population
7,11,0.000326,0.000000,0.001600,0.009384,0.027920,0.063127,0.061018,0.732015,0.104610,Full_Population


In [269]:
pop_wtd_ttm_sorted.to_excel("//rasscvo/DIAC/DIAC/Farm_Credit_System/AgWest/2024/PwC_Repl_Output/PD/pop_wtd_ttm_sorted.xlsx")

# Converting the portfolio level av g TTM to smoothed matrix : 

## Applying flooring :
#### A floor of 0.05% is applied to all transitions to a non-passing or Default rating

In [273]:
pop_wtd_ttm_floored=pop_wtd_ttm_sorted.copy()
pop_wtd_ttm_floored['11'] = pop_wtd_ttm_floored['11'].apply(lambda x: x if x > 0.0005 else 0.0005)  
pop_wtd_ttm_floored['Default'] = pop_wtd_ttm_floored['Default'].apply(lambda x: x if x > 0.0005 else 0.0005)  
pop_wtd_ttm_floored

,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment
0,Low Risk,0.817340,0.116970,0.040001,0.018267,0.002910,0.003143,0.000766,0.000500,0.000500,Full_Population
1,5,0.036077,0.792832,0.111087,0.041826,0.012501,0.002844,0.001779,0.000550,0.000504,Full_Population
2,6,0.009023,0.074051,0.749675,0.113385,0.036202,0.010484,0.003763,0.002630,0.000787,Full_Population
3,7,0.003328,0.020233,0.085470,0.762989,0.087591,0.024666,0.007912,0.005524,0.002288,Full_Population
4,8,0.001250,0.008110,0.032735,0.137167,0.689681,0.086040,0.027388,0.015771,0.001857,Full_Population
5,9,0.000141,0.005265,0.019534,0.046942,0.106801,0.704820,0.062924,0.038517,0.015056,Full_Population
6,10,0.000449,0.000343,0.004201,0.019583,0.104911,0.140472,0.511698,0.195508,0.022835,Full_Population
7,11,0.000326,0.000000,0.001600,0.009384,0.027920,0.063127,0.061018,0.732015,0.104610,Full_Population


## Geometric Interpolation:

Geometric interpolation is performed in the Default column values for all initial PD ratings among passing states. The pre-smoothing matrix does not show clear separation and rank-ordering in the Default column. For example, the default rate of PD rating 8 is less than that of PD rating 7. To adjust for this and produce a matrix that has default rates aligned with the intuition of the PD ratings, geometric interpolation is applied on the range of all passing initial PD ratings, such that all PD ratings between the floored 0.05% for Low Risk and the 1.51% for PD rating 9 are geometrically interpolated. This causes an uptick in the default rate for PD ratings 5-8 with the largest increase coming in PD rating 8. (This has alsobeen performed in a separate excel file named "creating_smoothed_single_matrix.xlsx" by the model development team. 

In [277]:
import math
pop_wtd_ttm_geom_intrplt=pop_wtd_ttm_floored.copy()
factor= math.pow(pop_wtd_ttm_geom_intrplt.iloc[5,9]/pop_wtd_ttm_geom_intrplt.iloc[0,9],1/5)
print(factor)

pop_wtd_ttm_geom_intrplt['Default'].iloc[1]=factor*pop_wtd_ttm_floored['Default'].iloc[0]
pop_wtd_ttm_geom_intrplt['Default'].iloc[2]=factor*pop_wtd_ttm_geom_intrplt['Default'].iloc[1]
pop_wtd_ttm_geom_intrplt['Default'].iloc[3]=factor*pop_wtd_ttm_geom_intrplt['Default'].iloc[2]
pop_wtd_ttm_geom_intrplt['Default'].iloc[4]=factor*pop_wtd_ttm_geom_intrplt['Default'].iloc[3]

pop_wtd_ttm_geom_intrplt

1.9758096394524038


C:\Users\dghatak005\AppData\Local\Temp\ipykernel_1448\82344349.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_wtd_ttm_geom_intrplt['Default'].iloc[1]=factor*pop_wtd_ttm_floored['Default'].iloc[0]
C:\Users\dghatak005\AppData\Local\Temp\ipykernel_1448\82344349.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_wtd_ttm_geom_intrplt['Default'].iloc[2]=factor*pop_wtd_ttm_geom_intrplt['Default'].iloc[1]
C:\Users\dghatak005\AppData\Local\Temp\ipykernel_1448\82344349.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

,initial_PD_rating,Low Risk,5,6,7,8,9,10,11,Default,Segment
0,Low Risk,0.817340,0.116970,0.040001,0.018267,0.002910,0.003143,0.000766,0.000500,0.000500,Full_Population
1,5,0.036077,0.792832,0.111087,0.041826,0.012501,0.002844,0.001779,0.000550,0.000988,Full_Population
2,6,0.009023,0.074051,0.749675,0.113385,0.036202,0.010484,0.003763,0.002630,0.001952,Full_Population
3,7,0.003328,0.020233,0.085470,0.762989,0.087591,0.024666,0.007912,0.005524,0.003857,Full_Population
4,8,0.001250,0.008110,0.032735,0.137167,0.689681,0.086040,0.027388,0.015771,0.007620,Full_Population
5,9,0.000141,0.005265,0.019534,0.046942,0.106801,0.704820,0.062924,0.038517,0.015056,Full_Population
6,10,0.000449,0.000343,0.004201,0.019583,0.104911,0.140472,0.511698,0.195508,0.022835,Full_Population
7,11,0.000326,0.000000,0.001600,0.009384,0.027920,0.063127,0.061018,0.732015,0.104610,Full_Population


## Adjust Diagonals so Rows add to 100%

Finally, to ensure the rows add back up to 100%, any increase in cells stemming from flooring and Geom Interpolation steps is offset by reducing the diagonal values by the same amount as the increase. 

In [278]:
pop_wtd_ttm_geom_intrplt

final_matrix=pop_wtd_ttm_geom_intrplt.drop('Segment',axis=1)
final_matrix.set_index('initial_PD_rating',inplace=True)

def adjust_transition_matrix(matrix):  
    n = matrix.shape[0]  # number of rows/columns in the matrix  
    adjusted_matrix = matrix.copy()  # create a copy of the original matrix  
      
    for i in range(n):  
        diagonal_element = matrix.iloc[i, i]  # get the diagonal element of the current row  
        non_diagonal_sum = np.sum(matrix.iloc[i, :]) - diagonal_element  # sum of non-diagonal elements  
          
        adjusted_diagonal_element = 1 - non_diagonal_sum  # adjust the diagonal element  
          
        adjusted_matrix.iloc[i, i] = adjusted_diagonal_element  # update the diagonal element in the adjusted matrix  
      
    return adjusted_matrix  



final_smoothed_pwc_ttm=adjust_transition_matrix(final_matrix)
final_smoothed_pwc_ttm

,Low Risk,5,6,7,8,9,10,11,Default
initial_PD_rating,,,,,,,,,
Low Risk,0.816944,0.116970,0.040001,0.018267,0.002910,0.003143,0.000766,0.000500,0.000500
5,0.036077,0.792348,0.111087,0.041826,0.012501,0.002844,0.001779,0.000550,0.000988
6,0.009023,0.074051,0.748509,0.113385,0.036202,0.010484,0.003763,0.002630,0.001952
7,0.003328,0.020233,0.085470,0.761420,0.087591,0.024666,0.007912,0.005524,0.003857
8,0.001250,0.008110,0.032735,0.137167,0.683919,0.086040,0.027388,0.015771,0.007620
9,0.000141,0.005265,0.019534,0.046942,0.106801,0.704820,0.062924,0.038517,0.015056
10,0.000449,0.000343,0.004201,0.019583,0.104911,0.140472,0.511698,0.195508,0.022835
11,0.000326,0.000000,0.001600,0.009384,0.027920,0.063127,0.061018,0.732015,0.104610


In [279]:
final_smoothed_pwc_ttm.to_excel("//rasscvo/DIAC/DIAC/Farm_Credit_System/AgWest/2024/PwC_Repl_Output/PD/final_smoothed_pwc_ttm.xlsx")